In [1]:
from typing import List

from typing_extensions import Annotated

from pydantic import BaseModel, ValidationError, field_validator
from pydantic.functional_validators import AfterValidator


def check_squares(v: int) -> int:
    assert v**0.5 % 1 == 0, f'{v} is not a square number'
    return v


def check_cubes(v: int) -> int:
    # 64 ** (1 / 3) == 3.9999999999999996 (!)
    # this is not a good way of checking cubes
    assert v ** (1 / 3) % 1 == 0, f'{v} is not a cubed number'
    return v


SquaredNumber = Annotated[int, AfterValidator(check_squares)]
CubedNumberNumber = Annotated[int, AfterValidator(check_cubes)]


class DemoModel(BaseModel):
    square_numbers: List[SquaredNumber] = []
    cube_numbers: List[CubedNumberNumber] = []

    @field_validator('square_numbers', 'cube_numbers', mode='before')
    def split_str(cls, v):
        if isinstance(v, str):
            return v.split('|')
        return v

    @field_validator('cube_numbers', 'square_numbers')
    def check_sum(cls, v):
        if sum(v) > 42:
            raise ValueError('sum of numbers greater than 42')
        return v


print(DemoModel(square_numbers=[1, 4, 9]))
#> square_numbers=[1, 4, 9] cube_numbers=[]
print(DemoModel(square_numbers='1|4|16'))
#> square_numbers=[1, 4, 16] cube_numbers=[]
print(DemoModel(square_numbers=[16], cube_numbers=[8, 27]))
#> square_numbers=[16] cube_numbers=[8, 27]
try:
    DemoModel(square_numbers=[1, 4, 2])
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    square_numbers.2
      Assertion failed, 2 is not a square number
    assert ((2 ** 0.5) % 1) == 0 [type=assertion_error, input_value=2, input_type=int]
    """

try:
    DemoModel(cube_numbers=[27, 27])
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    cube_numbers
      Value error, sum of numbers greater than 42 [type=value_error, input_value=[27, 27], input_type=list]
    """

square_numbers=[1, 4, 9] cube_numbers=[]
square_numbers=[1, 4, 16] cube_numbers=[]
square_numbers=[16] cube_numbers=[8, 27]
1 validation error for DemoModel
square_numbers.2
  Assertion failed, 2 is not a square number [type=assertion_error, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/assertion_error
1 validation error for DemoModel
cube_numbers
  Value error, sum of numbers greater than 42 [type=value_error, input_value=[27, 27], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


In [29]:
from typing import Any, List

from typing_extensions import Annotated

from pydantic import BaseModel, ValidationError
from pydantic.functional_validators import AfterValidator, BeforeValidator


def check_squares(v: int) -> int:
    print("check_squares")
    assert v**0.5 % 1 == 0, f'{v} is not a square number'
    return v


def double(v: Any) -> Any:
    print("double")
    return v * 2


# MyNumber = Annotated[int, AfterValidator(check_squares), AfterValidator(double)]


class DemoModel(BaseModel):
    number: List[Annotated[int, AfterValidator(check_squares), AfterValidator(double)]]


# print(DemoModel(number=[2, 8]))
# number=[4, 16]
try:
    demo = DemoModel(number=[4, 16])
    print(demo.number)
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    number.1
      Assertion failed, 8 is not a square number
    assert ((8 ** 0.5) % 1) == 0 [type=assertion_error, input_value=4, input_type=int]
    """

double
check_squares
double
check_squares
2 validation errors for DemoModel
number.0
  Assertion failed, 8 is not a square number [type=assertion_error, input_value=8, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/assertion_error
number.1
  Assertion failed, 32 is not a square number [type=assertion_error, input_value=32, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/assertion_error


In [32]:
from typing import Any, List

from typing_extensions import Annotated

from pydantic import BaseModel, ValidationError
from pydantic.functional_validators import AfterValidator, BeforeValidator


def convert_to_int(v: str) -> int:
    print("convert_to_int")
    assert int(v), f'{v} is not an integer'
    return v


def double(v: Any) -> Any:
    print("double")
    return v * 2


# MyNumber = Annotated[int, AfterValidator(check_squares), AfterValidator(double)]


class DemoModel(BaseModel):
    number: List[Annotated[int, BeforeValidator(convert_to_int), AfterValidator(double)]]


# print(DemoModel(number=[2, 8]))
# number=[4, 16]
try:
    demo = DemoModel(number=["aa", 16])
    print(demo.number)
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    number.1
      Assertion failed, 8 is not a square number
    assert ((8 ** 0.5) % 1) == 0 [type=assertion_error, input_value=4, input_type=int]
    """

convert_to_int
convert_to_int
double
1 validation error for DemoModel
number.0
  Value error, invalid literal for int() with base 10: 'aa' [type=value_error, input_value='aa', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


In [35]:
import json
from typing import Any, List

from typing_extensions import Annotated

from pydantic import (
    BaseModel,
    ValidationError,
    ValidationInfo,
    ValidatorFunctionWrapHandler,
)
from pydantic.functional_validators import WrapValidator


def maybe_strip_whitespace(
    v: Any, handler: ValidatorFunctionWrapHandler, info: ValidationInfo
) -> int:
    print(f"handler {handler}")
    print(f"info {info}")
    if info.mode == 'json':
        assert isinstance(v, str), 'In JSON mode the input must be a string!'
        # you can call the handler multiple times
        try:
            return handler(v)
        except ValidationError:
            return handler(v.strip())
    assert info.mode == 'python'
    assert isinstance(v, int), 'In Python mode the input must be an int!'
    # do no further validation
    return v


MyNumber = Annotated[int, WrapValidator(maybe_strip_whitespace)]


class DemoModel(BaseModel):
    number: List[MyNumber]


print(DemoModel(number=[2, 8]))
#> number=[2, 8]
print(DemoModel.model_validate_json(json.dumps({'number': [' 2 ', '8']})))
#> number=[2, 8]
try:
    DemoModel(number=['2'])
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    number.0
      Assertion failed, In Python mode the input must be an int!
    assert False
     +  where False = isinstance('2', int) [type=assertion_error, input_value='2', input_type=str]
    """

handler ValidatorCallable(Int(IntValidator { strict: false }))
info ValidationInfo(config={'title': 'DemoModel'}, context=None, data={}, field_name='number')
handler ValidatorCallable(Int(IntValidator { strict: false }))
info ValidationInfo(config={'title': 'DemoModel'}, context=None, data={}, field_name='number')
number=[2, 8]
handler ValidatorCallable(Int(IntValidator { strict: false }))
info ValidationInfo(config={'title': 'DemoModel'}, context=None, data={}, field_name='number')
handler ValidatorCallable(Int(IntValidator { strict: false }))
info ValidationInfo(config={'title': 'DemoModel'}, context=None, data={}, field_name='number')
number=[2, 8]
handler ValidatorCallable(Int(IntValidator { strict: false }))
info ValidationInfo(config={'title': 'DemoModel'}, context=None, data={}, field_name='number')
1 validation error for DemoModel
number.0
  Assertion failed, In Python mode the input must be an int! [type=assertion_error, input_value='2', input_type=str]
    For further informa

In [33]:
d

DemoModel(number=[8, 32])